In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="openai/gpt-oss-120b"
)

response = llm.invoke("The about cr7")
print(response.content)

**Cristiano Ronaldo — The Man Behind “CR7”**

---

## Quick Snapshot
| Item | Details |
|------|---------|
| **Full name** | Cristiano Ronaldo dos Santos Aveiro |
| **Born** | 5 Feb 1985 – Funchal, Madeira, Portugal |
| **Height** | 1.87 m (6 ft 2 in) |
| **Position** | Forward (mainly left‑wing/centre‑forward) |
| **Current club (2025)** | **Al Nassr FC** (Saudi Pro League) – jersey #7 |
| **National team** | Portugal (2003‑present) – captain |
| **Nickname** | “CR7” (C = Cristiano, R = Ronaldo, 7 = shirt number) |
| **Social‑media reach** | > 600 M followers across Instagram, Facebook & Twitter – one of the most‑followed athletes on the planet. |

---

## Career Timeline

| Years | Club | Appearances* | Goals* | Major Trophies |
|-------|------|--------------|--------|----------------|
| **2002‑2003** | Sporting CP (Portugal) | 31 | 5 | Primeira Liga (2002) – youth debut |
| **2003‑2009** | Manchester United (ENG) | 292 | 118 | 3 × Premier League, 1 × FA Cup, 2 × League Cup, 1 × UEFA

In [2]:
!pip install langchain_community

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.ibm.com/in-en/careers/search?field_keyword_18%5B0%5D=Entry%20Level")
page_data = loader.load().pop().page_content
print(page_data[:1000])  # print first 1000 chars safely















Search jobs - IBM Careers























































































Home





Careers




Search





                                    



  
    Let's find you the right job






    


                                





Can't find a job that would be a good fit? Join our talent network or set up a job alert and we'll keep you up to date with career opportunities!
 































































































In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}

    ### INSTRUCTION:
    The scraped text is from a careers/jobs page.
    Extract ALL job postings you find and return them strictly in this JSON format:

    [
      {{
        "role": "",
        "experience": "",
        "skills": [],
        "description": ""
      }}
    ]

    RULES:
    - MUST return only valid JSON.
    - skills must be a list.
    - No explanation, no text, only JSON.
    """
)

In [13]:
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="openai/gpt-oss-120b"
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke({"page_data": page_data})
print(res.content)


[
]


In [14]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [15]:
import chromadb
import uuid


client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas=[{"links": row["Links"]}],
            ids=[str(uuid.uuid4())]
        )


In [16]:
jobs = ["java", "python"]

links = [
    m.get("links")
    for meta_list in collection.query(
        query_texts=jobs,
        n_results=2
    ).get("metadatas", [])
    for m in meta_list
]

links

['https://example.com/java-portfolio',
 'https://example.com/android-portfolio',
 'https://example.com/ml-python-portfolio',
 'https://example.com/python-portfolio']

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm

# Generate email for each job
for job in jobs:
    print(f"\n{'='*50}\nGenerating email for: {job}\n{'='*50}\n")
    res = chain_email.invoke({"job_description": job, "link_list": links})
    print(getattr(res, 'content', res))


Generating email for: java

Subject: Accelerate Your Java Projects with AtliQ’s Expert Solutions  

Hi [Client Name],

I’m Mohan, Business Development Executive at AtliQ, an AI‑driven software consulting firm that helps enterprises streamline operations through custom, high‑performance applications.

**Why AtliQ for your Java needs?**  
- **Deep Java expertise:** Our team has delivered end‑to‑end Java solutions ranging from micro‑services architectures to large‑scale enterprise platforms.  
- **AI‑enhanced automation:** We embed intelligent automation to reduce manual effort, improve reliability, and cut operational costs.  
- **Scalable, secure code:** Leveraging modern frameworks (Spring Boot, Quarkus, Jakarta EE) and DevSecOps pipelines, we ensure your applications grow with your business while staying secure.  
- **Rapid delivery:** Agile methodology and automated testing accelerate time‑to‑market without compromising quality.

**Relevant portfolio highlights**  
- **Java Projects